In [ ]:
import numpy as np
from scipy.optimize import fsolve, minimize_scalar
from scipy.constants import k, e
from scipy.special import lambertw

T = 300  # K
q = e  # C
Vt = k * T / q  # Thermal voltage, V

Jph_mA_cm2 =  26.3 # mA/cm²
Jph = Jph_mA_cm2 / 1000  # A/cm²
J0_mA_cm2 = 1.165176e-13 # mA/cm²
J0 = J0_mA_cm2 / 1000  # A/cm²
Rs = 0.8156  # Series resistence, Ω·cm²
Rsh = 2853 # Shunt resistence, Ω·cm²
n = 1.349362  # ideal factor

In [ ]:
def explicit_current_density(V, Jph, J0, Rs, Rsh, n):
    w = Rs * J0 * Rsh * np.exp(Rsh * (Rs * Jph + Rs * J0 + V) /
                               (n * Vt * (Rs + Rsh))) / (n * Vt * (Rs + Rsh))
    return (-V / (Rs + Rsh) - lambertw(w) * n * Vt / Rs + Rsh * (J0 + Jph) /
            (Rs + Rsh)).real


def explicit_voltage(J, Jph, J0, Rs, Rsh, n):
    w = J0 * Rsh * np.exp(Rsh * (-J + Jph + J0) / (n * Vt)) / (n * Vt)
    return (-J * Rs - J * Rsh + Jph * Rsh - n * Vt * lambertw(w) +
            J0 * Rsh).real


def Jsc(Jph, J0, Rs, Rsh, n):
    w = Rs * Rsh * J0 * np.exp(Rsh * (Rs * Jph + Rs * J0) /
                               (n * Vt * (Rs + Rsh))) / (n * Vt * (Rs + Rsh))
    return (-lambertw(w) * n * Vt / Rs + Rsh * (Jph + J0) / (Rs + Rsh)).real


def Voc(Jph, J0, Rs, Rsh, n):
    exponent = Rsh * (Jph + J0) / (n * Vt)
    if exponent > 700:  # appoximation for large exponent
        L1 = np.log(J0 * Rsh / (n * Vt)) + exponent  # ln(w)
        L2 = np.log(L1)  # ln(ln(w))
        W0 = L1 - L2

        def lambert_literate(w):
            # ln(w) + w = L1
            f = np.log(w) + w - L1
            df = 1.0 / w + 1.0
            delta = f / df
            return w - delta

        # newton iteration
        W = W0
        for _ in range(20):
            W_next = lambert_literate(W)
            # threshold for convergence
            if np.abs(W_next - W) < 1e-10:
                W = W_next
                break
            W = W_next
        W = W.real
    else:
        w = J0 * Rsh * np.exp(exponent) / (n * Vt)
        W = lambertw(w).real
    return (Jph * Rsh - n * Vt * W + J0 * Rsh).real


print(Jsc(Jph, J0, Rs, Rsh, n))
print(Voc(Jph, J0, Rs, Rsh, n))

0.026292483648908405
1.1523771539088783
